## Environment

In [1]:
!source /etc/network_turbo
!echo 'export HF_ENDPOINT=https://hf-mirror.com' >> ~/.bashrc
!source ~/.bashrc

设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [2]:
import os
# 强制使用本地文件（避免联网尝试）
os.environ["TRANSFORMERS_OFFLINE"] = "1" 

In [3]:
!pip install timm
!pip install opencv-python pandas albumentations

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [4]:
import cv2 # imread方法不支持读取带中文路径的文件，但是可以用imdecode方法(较麻烦）
# import gc
import numpy as np
import pandas as pd
import itertools
from tqdm.autonotebook import tqdm
import albumentations as A
import matplotlib.pyplot as plt

import pdb
# pdb.set_trace() # 打断点

import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer
from safetensors.torch import load_file

print("OK")

/tmp/ipykernel_2840/3561960892.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


OK


## Config

*A note on config and CFG: I wrote the codes with python scripts and then converted it into a Jupyter Notebook. So, in case of python scripts, config is a normal python file where I put all the hyperparameters and in the case of Jupyter Notebook, its a class defined in the beginning of the notebook to keep all the hyperparameters.*

In [5]:
class SelectDataset:
    basePath = "/root/autodl-tmp/data/"
    flickr8k = "flickr8k"
    flickr30k = "flickr30k"
    ourDataset = "ourDataset"

In [6]:
class CFG:
    debug = False
    useTripletMarginLoss = True # TODO: 是否使用三元组损失
    margin = 2 # TODO:
    alpha = 0.5 # TODO:
    selectDataset =  SelectDataset.ourDataset # TODO: 选择在哪个数据集进行训练    
    selectDatasetPath = SelectDataset.basePath + selectDataset
    image_path = selectDatasetPath + "/images"
    captions_path = selectDatasetPath

    batch_size = 4 # TODO: 因为是按批训练和对比，所以效果很差
    num_workers = 4
    head_lr = 1e-3
    image_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    weight_decay = 1e-3
    patience = 1
    factor = 0.8
    epochs = 30 # TODO: adjust
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model_name = '../autodl-tmp/model/resnet50'
    image_embedding_dim = 1000 # 2048
    text_encoder_model = "../autodl-tmp/model/distilbert-base-uncased"
    text_embedding_dim = 768
    text_tokenizer = "../autodl-tmp/model/distilbert-base-uncased"
    max_length = 200

    pretrained = True # for both image encoder and text encoder
    trainable = True # for both image encoder and text encoder
    temperature = 1.0

    # image size
    size = 224

    # for projection head; used for both image and text encoders
    num_projection_layers = 1
    projection_dim = 256 
    dropout = 0.1

## Some Pre-processsing

### For dataset Flickr 30k
> https://www.kaggle.com/datasets/hsankesara/flickr-image-dataset

1. 导入环境变量(覆盖kaggle默认的存储位置)
```bash
export KAGGLEHUB_CACHE="./autodl-tmp"
echo $KAGGLEHUB_CACHE
```
2. 执行下面的python脚本（随便放到一个.py文件然后执行即可）：
```python
import kagglehub
# 下载数据集
path = kagglehub.dataset_download("hsankesara/flickr-image-dataset")
print("Path to dataset  Flickr 30k:", path)  #
```

In [7]:
if CFG.selectDataset == SelectDataset.flickr30k:
    df = pd.read_csv(f"{CFG.captions_path}/captions.csv",  delimiter="|")
    df.columns = ['image', 'caption_number', 'caption']
    df['caption'] = df['caption'].str.lstrip()
    df['caption_number'] = df['caption_number'].str.lstrip()
    print(f"长度为{len(df)}, \n某个元素如下:\n{df.loc[len(df)-1, :]}")
    df.loc[19999, 'caption_number'] = "4"
    df.loc[19999, 'caption'] = "A dog runs across the grass ."
    ids = [id_ for id_ in range(len(df) // 5) for i in range(5)]
    df['id'] = ids
    df.head()


### For dataset Flickr 8k

In [8]:
if CFG.selectDataset == SelectDataset.flickr8k:
    df = pd.read_csv(f"{CFG.captions_path}/captions.csv",  delimiter=",")
    df.columns = ['image', 'caption']
    df['caption'] = df['caption'].str.lstrip()
    
    print(f"长度为{len(df)}, \n某个元素如下:\n{df.loc[len(df)-1, :]}")
    ids = [id_ for id_ in range(len(df) // 5) for i in range(5)]
    df['id'] = ids
    df.head()

### For dataset collected by ours

In [9]:
if CFG.selectDataset == SelectDataset.ourDataset: 
    df = pd.read_csv(f"{CFG.captions_path}/captions.csv",  delimiter=",", header=None)
    df.columns = ['image', 'caption']
    df['caption'] = df['caption'].str.lstrip()
    print(df['image'][0])
    print(df['caption'][0])
    
    print(f"长度为{len(df)}, \n某个元素如下:\n{df.loc[len(df)-1, :]}")
    ids = [id_ for id_ in range(len(df))]
    df['id'] = ids
    df.head()

pic (1).jpg
A close-up of the head of a vulture, showing its white head, yellow beak, and sharp eyes. Its expression appears very majestic.
长度为287, 
某个元素如下:
image                                          pic (287).jpg
caption    A pile of yellow pumpkins and zucchinis with s...
Name: 286, dtype: object


## Utils

In [10]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()        

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):    
        # pdb.set_trace()
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]


## Dataset

As you can see in the tittle image of this article, we need to encode both images and their describing texts. So, the dataset needs to **return both images and texts**. Of course we are not going to feed raw text to our text encoder! We will use **DistilBERT** model (which is smaller than BERT but performs nearly as well as BERT) from **HuggingFace** library as our text encoder; so, we need to **tokenize** the sentences (captions) with DistilBERT tokenizer and then feed the token ids (input_ids) and the attention masks to DistilBERT. Therefore, the dataset needs to take care of the tokenization as well. Below you can see the dataset's code. Below that I'll explain the most important things that is happening in the code.

In the **\_\_init\_\_** we receive a tokenizer object which is actually a HuggingFace tokinzer; this tokenizer will be loaded when running the model. We are padding and truncating the captions to a specified max_length. In the **\_\_getitem\_\_** we will first load an encoded caption which is a dictionary with keys input_ids and attention_mask, make tensors out of its values and after that we will load the corresponding image, transform and augment it (if there is any!) and then we make it a tensor and put it in the dictionary with "image" as the key. Finally we put the raw text of the caption with the key "caption" in the dictionary only for visualization purposes. 

I did not use additional data augmentations but you can add them if you want to improve the model's performance.

In [11]:
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """

        self.image_filenames = image_filenames
        self.captions = list(captions)
        self.encoded_captions = tokenizer(
            list(captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transforms = transforms

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }

        image = cv2.imread(f"{CFG.image_path}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # image = Image.open(f"{CFG.image_path}/{self.image_filenames[idx]}").convert('RGB')        
        image = self.transforms(image=image)['image']
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['caption'] = self.captions[idx]

        return item


    def __len__(self):
        return len(self.captions)



def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size),
                A.Normalize(max_pixel_value=255.0),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size),
                A.Normalize(max_pixel_value=255.0),
            ]
        )

## Image Encoder

The image encoder code is straight forward. I'm using PyTorch Image Models library (timm) here which makes a lot of different image models available from ResNets to EfficientNets and many more. Here we will use a ResNet50 as our image encoder. You can easily use torchvision library to use ResNets if you don't want to install a new library.

The code encodes each image to a fixed size vector with the size of the model's output channels (in case of ResNet50 the vector size will be **2048**). This is the output after the nn.AdaptiveAvgPool2d() layer.

In [12]:
class ImageEncoder(nn.Module):
    """
    Encode images to a fixed size vector
    """

    def __init__(
        self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable
    ):
        super().__init__()
        
        # 加载模型权重
        self.model = timm.create_model(
            model_name, pretrained=False, global_pool="avg"
        )
        if pretrained:
            state_dict = load_file(f"{model_name}/model.safetensors")
            self.model.load_state_dict(state_dict)        
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

## Text Encoder

As I mentioned before, I'll use DistilBERT as the text encoder. Like its bigger brother BERT, two special tokens will be added to the actual input tokens: **CLS** and **SEP** which mark the start and end of a sentence. To grab the whole representation of a sentence (as the related BERT and DistilBERT papers point out) we use the final representations of the CLS token and we hope that this representation captures the overall meaning of the sentence (caption). Thinking it in this way, it is similar to what we did to images and converted them into a fixed size vector.

In the case of DistilBERT (and also BERT) the output hidden representation for each token is a vector with size **768**. So, the whole caption will be encoded in the CLS token representation whose size is 768.

In [13]:
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name, local_files_only=True)  # 确保只使用本地文件
        else:
            self.model = DistilBertModel(config=DistilBertConfig())
            
        for p in self.model.parameters():
            p.requires_grad = trainable

        # we are using the CLS token hidden representation as the sentence's embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

## Projection Head

I used [Keras code example implementation](https://keras.io/examples/nlp/nl_image_search/) of projection head to write the following in PyTorch.
Now that we have encoded both our images and texts into fixed size vectors (2048 for image and 768 for text) we need to bring (project) them into a **new world** (!) with **similar dimensions** for both images and texts in order to be able to compare them and push apart the non-relevant image and texts and pull together those that match. So, the following code will bring the 2048 and 768 dimensional vectors into a 256 (projection_dim) dimensional world, where we can **compare** them.

"embedding_dim" is the size of the input vector (2048 for images and 768 for texts) and "projection_dim" is the the size of the output vector which will be 256 for our case. For understanding the details of this part you can refer to the CLIP paper.

In [14]:
class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=CFG.projection_dim, # output_dim
        dropout=CFG.dropout
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)        
    
    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)

        x = F.softmax(x, dim=-1) # 加层softmax，这样矩阵叉乘 才能 等于 余弦相似度
        return x

## CLIP

This part is where all the fun happens! I'll also talk about the loss function here. I translated some of the code from Keras code examples into PyTorch for writing this part. Take a look at the code and then read the explanation below this code block.

Here we will use the previous modules that we built to implement the main model. The \_\_init\_\_ function is self-explanatory. In the forward function, we first encode the images and texts separately into fixed size vectors (with different dimensionalities). After that, using separate projection modules we project them to that shared world (space) that I talked about previously. Here the encodings will become of similar shape (256 in our case). After that we will compute the loss. Again I recommend reading CLIP paper to get it better but I'll try my best to explain this part.

In **Linear Algebra**, one common way to measure if two vectors are of similar characteristics (they are like each other) is to calculate their **dot product** (multiplying the matching entries and take the sum of them); if the final number is big, they are alike and if it is small they are not (relatively speaking)!

Okay! What I just said is the most important thing to have in mind to understand this loss function. Let's continue. We talked about two vectors, but, what do we have here? We have image_embeddings, a matrix with shape (batch_size, 256) and text_embeddings with shape (batch_size, 256). Easy enough! it means we have two groups of vectors instead of two single vectors. How do we measure how similar two groups of vectors (two matrices) are to each other? Again, with dot product (@ operator in PyTorch does the dot product or matrix multiplication in this case). To be able to multiply these two matrices together, we transpose the second one. Okay, we get a matrix with shape (batch_size, batch_size) which we will call logits. (temperature is equal to 1.0 in our case, so, it does not make a difference. You can play with it and see what difference it makes. Also look at the paper to see why it is here!).

I hope you are still with me! If not it's okay, just review the code and check their shapes. Now that we have our logits, we need targets. I need to say that there is a more straight forward way to obtain targets but I had to do this for our case (I'll talk about why in a next paragraph).

Let's consider what we hope that this model learns: **we want it to learn "similar representations (vectors)" for a given image and the caption describing it. Meaning that either we give it an image or the text describing it, we want it to produce same 256 sized vectors for both.**

#### Check the cell below this code block for the continue of the explanations

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding_dim=CFG.image_embedding_dim,        
        text_embedding_dim=CFG.text_embedding_dim,        
        use_triplet: bool = False,
        margin: float = CFG.margin,
        alpha: float = CFG.alpha,
    ):
        super().__init__()
        self.image_encoder    = ImageEncoder()
        self.text_encoder     = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding_dim) # input_dim
        self.text_projection  = ProjectionHead(embedding_dim=text_embedding_dim) # input_dim
        self.temperature      = temperature

        # 三元组损失相关
        self.use_triplet      = use_triplet
        self.margin           = margin
        self.alpha            = alpha
        if use_triplet:
            self.triplet_loss_fn = nn.TripletMarginLoss(
                margin=margin, p=2, reduction='mean'
            )

    def forward(self, batch, text_embeddings, image_embeddings):        
        # 1. 编码并投影
        image_feats = self.image_encoder(batch["image"])
        text_feats  = self.text_encoder(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"]
        )
        image_embs = self.image_projection(image_feats)  # (B, D)
        text_embs  = self.text_projection(text_feats)    # (B, D)

        # 2. 计算原始 CLIP 对比损失（双向 InfoNCE）
        logits1 = (text_embs @ image_embeddings.T) / self.temperature  # (B, A)
        logits2 = (image_embs @ text_embeddings.T) / self.temperature  # (B, A)

        sim_i2i = image_embs @ image_embeddings.T # (B, A)
        sim_t2t = text_embs  @ text_embeddings.T # (B, A)
        targets  = (sim_i2i + sim_t2t) / 2 * self.temperature # 类似于CrossEntropy函数中的label是概率

        texts_loss  = cross_entropy(logits1, targets, reduction='mean') # 标量
        images_loss = cross_entropy(logits2, targets, reduction='mean')
        base_loss = (texts_loss + images_loss) / 2.0

        # 3. 如果不使用三元组损失，直接返回
        if not self.use_triplet:
            return base_loss

        # 4. 构造 Hard Negatives
        #    对角线置 -inf，再取每行最大相似度对应的 idx 和 每列最大相似度对应的 idy        
        sim1 = image_embs @ text_embeddings.T     # (B, A)，且A>=B
        sim1.fill_diagonal_(-1e9)
        hardest_neg_idx = sim1.argmax(dim=1)  # 形状为(B,) 对图片最难区分的文本负样本

        sim2 = text_embs @ image_embeddings.T  # (B, A)
        hardest_neg_idy = sim2.argmax(dim=1) # 形状为(,B), 对文本最难区分的图片负样本

        neg_text_embs = text_embeddings[hardest_neg_idx]  # (B, D)
        neg_image_embs = image_embeddings[hardest_neg_idy] # (B, D)
        
        # 5. 计算三元组损失，
        # 这里选取对image最难区分的负样本（text）。其中anchor=image_embs, positive=text_embs, negative=neg_text_embs
        triplet_loss1 = self.triplet_loss_fn(
            image_embs, text_embs, neg_text_embs
        )
        # 同理
        triplet_loss2 = self.triplet_loss_fn(
            text_embs, image_embs, neg_image_embs
        )
        # 图片负样本和文本负样本都需要考虑
        triplet_loss = (triplet_loss1 + triplet_loss2) / 2.0
        
        # 6. 融合
        total_loss = base_loss + self.alpha * triplet_loss
        return total_loss, base_loss

    def set_complete_embeddings(self, complete_image_embeddings, complete_text_embeddings):
        image_embeddings = complete_image_embeddings
        text_embeddings = complete_text_embeddings

def cross_entropy(preds, targets, reduction='none', isTop5=True):    
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

    


So, in the best case scenario, text_embeddings and image_embedding matricies should be the same because they are describing similar things. Let's think now: if this happens, what would the logits matrix be like? Let's see with a simple example!

In [16]:
# A simple Example

batch_size = 4
dim = 256
embeddings = torch.randn(batch_size, dim)
out = embeddings @ embeddings.T
print(F.softmax(out, dim=-1))

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


So logits, in the best case, will be a matrix that if we take its softmax, will have 1.0s in the diagonal (An identity matrix to call it with fancy words!). As the loss function's job is to make model's predictions similar to targets (at least in most cases!), we want such a matrix as our target. That's the reason why we are calculating images_similarity and texts_similarity matrices in the code block above.

Now that we've got our targets matrix, we will use simple cross entropy to calculate the actual loss. I've written the full matrix form of cross entropy as a function which you can see in the bottom of the code block. Okay! We are done! Wasn't it simple?! Alright, you can ignore the next paragraph but if you are curious, there is an important note in that.

**Here's why I didn't use a simpler approach**: I need to admit that there's a simpler way to calculate this loss in PyTorch; by doing this: nn.CrossEntropyLoss()(logits, torch.arange(batch_size)). Why I did not use it here? For 2 reasons. 1- The dataset we are using has multiple captions for a single image; so, there is the possibility that two identical images with their similar captions exist in a batch (it is rare but it can happen). Taking the loss with this easier method will ignore this possibility and the model learns to pull apart two representations (assume them different)  that are actually the same. Obviously, we don't want this to happen so I calculated the whole target matrix in a way that takes care of these edge cases. 2- Doing it the way I did, gave me a better understanding of what is happening in this loss function; so, I thought it would give you a better intuition as well!

## Train

### load train and valid dataloaders
Here are some funtions to help us load train and valid dataloaders, our model and then train and evaluate our model on those. There's not much going on here; just simple training loop and utility functions

In [17]:
def make_train_valid_dfs():
    dataframe = df # 预处理时已经赋值     
    max_id = dataframe["id"].max() + 1 if not CFG.debug else 100
    image_ids = np.arange(0, max_id) # 防止同一张图片的不同caption分散到训练集和测试集
    np.random.seed(42)
    valid_ids = np.random.choice(
        image_ids, size=int(0.2 * len(image_ids)), replace=False
    )
    train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    train_dataframe = dataframe[dataframe["id"].isin(train_ids)].reset_index(drop=True)
    valid_dataframe = dataframe[dataframe["id"].isin(valid_ids)].reset_index(drop=True)
    return train_dataframe, valid_dataframe


def build_loaders(dataframe, tokenizer, mode):
    transforms = get_transforms(mode=mode)
    dataset = CLIPDataset(
        dataframe["image"].values,
        dataframe["caption"].values,
        tokenizer=tokenizer,
        transforms=transforms,
    )
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers,
        shuffle = False
        # shuffle=True if mode == "train" else False,
    )
    return dataloader

### 用于训练阶段：分别按批次加载完整的text和image embeddings

In [18]:
def get_complete_text_embeddings(df, model):
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)  #
    loader = build_loaders(df, tokenizer, mode="train")            
    
    text_embeddings = []
    
    for batch in tqdm(loader):
        text_feats = model.text_encoder(
            input_ids=batch["input_ids"].to(CFG.device),
            attention_mask=batch["attention_mask"].to(CFG.device)
        )
        text_embs  = model.text_projection(text_feats)    # (B, D)
        text_embeddings.append(text_embs)
    return torch.cat(text_embeddings)


def get_complete_image_embeddings(df, model):
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)  #
    loader = build_loaders(df, tokenizer, mode="train")   
    
    image_embeddings = []

    for batch in tqdm(loader):
        image_features = model.image_encoder(batch["image"].to(CFG.device))
        image_embs = model.image_projection(image_features)
        image_embeddings.append(image_embs)
    return torch.cat(image_embeddings)

Here's a handy function to train our model. There's not much happening here; just loading the batches, feeding them to the model and stepping the optimizer and lr_scheduler.

In [ ]:
def train_epoch(model, train_loader, train_df, optimizer, lr_scheduler, step):
    # 训练时要用到
    complete_text_embeddings = get_complete_text_embeddings(train_df, model)  
    complete_image_embeddings = get_complete_image_embeddings(train_df, model)
        
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:                
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        if CFG.useTripletMarginLoss:
            loss, _ = model(batch, complete_text_embeddings, complete_image_embeddings)
        else:
            loss = model(batch, complete_text_embeddings, complete_image_embeddings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()

        count = batch["image"].size(0)
        # pdb.set_trace()
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter

def draw_train_loss(x, y, name):
    plt.plot(x,y, 'r--*')
    plt.savefig(f"results/{name}.jpg")
    
    plt.show()    
    
def valid_epoch(model, valid_loader, isTotalLoss=False):
    loss_meter = AvgMeter()

    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        if CFG.useTripletMarginLoss:
            if isTotalLoss: # 取2种Loss融合之后的值
                base_loss, _ = model(batch, complete_text_embeddings, complete_image_embeddings)
            else:
                _, base_loss = model(batch, complete_text_embeddings, complete_image_embeddings)
        else:
            base_loss = model(batch, complete_text_embeddings, complete_image_embeddings)

        count = batch["image"].size(0)
        loss_meter.update(base_loss.item(), count)

        tqdm_object.set_postfix(valid_loss=loss_meter.avg)
    return loss_meter


def main():
    train_df, valid_df = make_train_valid_dfs()
    print("成功划分数据集")
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)  # 确保只使用本地文件)  
    print("成功加载分词器")
    train_loader = build_loaders(train_df, tokenizer, mode="train")
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")    

    model = CLIPModel(use_triplet=CFG.useTripletMarginLoss).to(CFG.device)          

    print("成功加载image和text的模型")
    params = [
        {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
        {"params": itertools.chain(
            model.image_projection.parameters(), model.text_projection.parameters()
        ), "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
    ]
    optimizer = torch.optim.AdamW(params, weight_decay=0.)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=CFG.patience, factor=CFG.factor
    )
    step = "epoch"

    best_loss = float('inf')
    train_loss_list = []
    suffix_name = f"dataset_{CFG.selectDataset}_useTriplet_{CFG.useTripletMarginLoss}"
    os.makedirs("best_models", mode=0o777, exist_ok=True) # 存储最佳模型
    os.makedirs("results", mode=0o777, exist_ok=True) # 存储结果图片
    os.makedirs("train_loss", mode=0o777, exist_ok=True) # 存储训练误差，用以对比不同方法
    for epoch in range(CFG.epochs):
        print(f"Epoch: {epoch + 1}")
        model.train()
        train_loss = train_epoch(model, train_loader, train_df, optimizer, lr_scheduler, step)        
        train_loss_list.append(train_loss.avg)
        model.eval()
        with torch.no_grad():
            valid_loss = valid_epoch(model, valid_loader, True) # 2种Loss(假如存在）
        
        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            torch.save(model.state_dict(), f"best_models/{suffix_name}_best.pt")
            print("Saved Best Model!")
        
        lr_scheduler.step(valid_loss.avg)
    # 保存图片
    draw_train_loss([i for i in range(1, CFG.epochs+1)], train_loss_list, suffix_name)
    
    # 写入loss到文件，方便后续与其他方法比对
    file_path = f"train_loss/{suffix_name}_train_loss.txt"
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(str(train_loss_list))

### Run
Running the next cell start training the model. Put the kernel on GPU mode. Every epoch should take about 24 minutes on GPU (even one epoch is enough!). It can take one minute before training actually starts because we are going to encode all the captions once in the train and valid dataset, so please don't stop it! Every thing is working fine.

In [20]:
%%time
print("开始训练")
main()

开始训练
成功划分数据集
成功加载分词器


  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

成功加载image和text的模型
Epoch: 1


  0%|          | 0/58 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacity of 23.57 GiB of which 90.81 MiB is free. Including non-PyTorch memory, this process has 23.47 GiB memory in use. Of the allocated memory 22.77 GiB is allocated by PyTorch, and 398.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### 读取train_loss以比较两个方法的不同

In [21]:
if 1 == 0:
    # 读取第一个文件
    suffix_name = f"dataset_{CFG.selectDataset}_useTriplet_False"
    file_path = f"{suffix_name}_train_loss.txt"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()  # 读取整个文件内容
        train_loss_list1 = eval(content)  # use one loss
        print("获取最后一个loss", train_loss_list1[len(train_loss_list1)-1])
    
    # 读取第二个文件
    suffix_name1 = f"dataset_{CFG.selectDataset}_useTriplet_True"
    file_path1 = f"{suffix_name1}_train_loss.txt"
    with open(file_path1, 'r', encoding='utf-8') as file:
        content = file.read()  # 读取整个文件内容
        train_loss_list2 = eval(content) # use two loss
        print("获取最后一个loss", train_loss_list2[len(train_loss_list2)-1])
    
    x = [i for i in range(1, len(train_loss_list1)+1)]
    # 绘制两条不同颜色的线
    plt.plot(x, train_loss_list1, color='blue', label='one loss')    # 蓝色线
    plt.plot(x, train_loss_list2, color='red', linestyle='--', label='two loss')  # 红色虚线
    # 添加标题和标签
    plt.title(f'In dataset {CFG.selectDataset}, compare two different method', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Train loss', fontsize=12)
    
    # 添加图例
    plt.legend()
    
    # 添加网格线
    plt.grid(alpha=0.3)
    
    # 显示图形
    plt.show()

## Inference

Okay! We are done with training the model. Now, we need to do inference which in our case will be giving the model a piece of text and want it to retrieve the most relevant images from an unseen validation (or test) set.

### Getting Image Embeddings

In this function, we are loading the model that we saved after training, feeding it images in validation set and returning the image_embeddings with shape (valid_set_size, 256) and the model itself.

In [22]:
def get_image_embeddings(valid_df, model_path):
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)  #
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")
    
    model = CLIPModel(use_triplet=CFG.useTripletMarginLoss).to(CFG.device)
    model.load_state_dict(torch.load(model_path, map_location=CFG.device))
    model.eval() # 禁用dropout
    
    valid_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image_features = model.image_encoder(batch["image"].to(CFG.device))
            image_embeddings = model.image_projection(image_features)
            valid_image_embeddings.append(image_embeddings)
    return model, torch.cat(valid_image_embeddings), valid_loader

### 加载最佳模型并计算最小损失值：

In [23]:
_, valid_df = make_train_valid_dfs()
model, image_embeddings, valid_loader = get_image_embeddings(valid_df, f"best_models/dataset_{CFG.selectDataset}_useTriplet_{CFG.useTripletMarginLoss}_best.pt")

if 1==1:
    with torch.no_grad():    
        smallest_valid_loss = valid_epoch(model, valid_loader, True) # 2种Loss(假如存在）
        print(smallest_valid_loss)
        smallest_valid_base_loss = valid_epoch(model, valid_loader, False) # INFONCE
        print(smallest_valid_base_loss)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.57 GiB of which 18.81 MiB is free. Including non-PyTorch memory, this process has 23.54 GiB memory in use. Of the allocated memory 22.86 GiB is allocated by PyTorch, and 382.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### 计算 Top_k err:

In [ ]:
def cal_count_for_topk_err(model, batch, batch_n, image_embeddings, topk=5): # 这里image_embeddings的形状为(A, D)，其中A表示整个测试集大小
    batch_start = batch_n * CFG.batch_size
    model.eval() # 禁用dropout
    # 按batch处理文本
    with torch.no_grad():
        text_feats = model.text_encoder(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"]
        )
        text_embs  = model.text_projection(text_feats)    # (B, D)
        image_embs = image_embeddings # (A, D)        
        
        # 这里没有必要采用L2范数归一化？
        # text_embeddings_n = F.normalize(text_embs, p=2, dim=-1)
        # image_embeddings_n = F.normalize(image_embs, p=2, dim=-1)        
        
        dot_similarity = text_embeddings_n @ image_embeddings_n.T # (B,A) ,A表示整个测试集大小   
    
    values, indices = torch.topk(dot_similarity.squeeze(0), k=topk, dim=-1)
    count = 0
    
    for i in range(0, len(indices)): #  B
        for j in range(len(indices[0])): # topk
            if indices[i][j] == i + batch_start:
                count += 1
                break            
    return count


def count_topk_err(model, image_embeddings, topk=5):
    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    total_count, batch_n = 0, 0
    for batch in tqdm_object:                
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}        
        total_count += cal_count_for_topk_err(model, batch, batch_n, image_embeddings, topk)        
        batch_n += 1
    # pdb.set_trace()
    topk_err = 1 - total_count / len(valid_df) # 相当于1-acc
    print(f"top {topk} err: {topk_err}")
        

In [ ]:
count_topk_err(model, image_embeddings, topk=5) # 计算topk_err

### Finding Matches

This function does the final task that we wished our model would be capable of: it gets the model, image_embeddings, and a text query. It will display the most relevant images from the validation set! Isn't it amazing? Let's see how it performs after all!

In [ ]:
def find_matches(model, image_embeddings, query, image_filenames, n=9):
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)  # 确保只使用本地文件)
    encoded_query = tokenizer([query])
    batch = {
        key: torch.tensor(values).to(CFG.device)
        for key, values in encoded_query.items()
    }
    with torch.no_grad():
        text_features = model.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        text_embeddings = model.text_projection(text_features)
    
    dot_similarity = text_embeddings_n @ image_embeddings_n.T
    
    values, indices = torch.topk(dot_similarity.squeeze(0), n * 5)
    matches = [image_filenames[idx] for idx in indices[::5]]
    
    _, axes = plt.subplots(3, 3, figsize=(10, 10))
    plt.suptitle(f"In dataset {CFG.selectDataset}, \n useTripletMarginLoss={CFG.useTripletMarginLoss}"+
                 "\nQuery: " +query)
    for match, ax in zip(matches, axes.flatten()):
        image = cv2.imread(f"{CFG.image_path}/{match}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)         
        ax.imshow(image)
        ax.axis("off")
    
    plt.show()

This is how we use this function. Aaaannnndddd the results:

In [ ]:
query = valid_df['caption'][0]# For example, the query is "one dog sitting on the grass"
print(query)
find_matches(model, 
             image_embeddings,
             query=query,
             image_filenames=valid_df['image'].values,
             n=9)

## Final words

I hope you have enjoyed this article. Implementing this paper was a really interesting experience for me. I want to thank Khalid Salama for the great Keras code example he provided which inspired me to write something similar in PyTorch.